本项目通过抓取武汉链家二手房在售信息进行保存并进行可视化分析，利用requests和BeautifulSoup库进行抓取解析，对于得到的数据利用pandas进行整理，后期将会利用matplotlib，seaborn进行可视化分析并总结。

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
import re

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}

In [3]:
#获取解析页面
def get_page(url,num):
    try:
        r = requests.get(url,headers=headers)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        print('--------第%d页抓取成功--------'%num) 
        return r.text
    except:
        return None

#解析页面
def parse_page(content):
    soup = BeautifulSoup(content,'lxml')
    pattern = re.compile('[0-9]{4}')  #利用正则匹配年份
    years = [i.get_text().strip().split('-')[0] for i in soup.select('div[class="positionInfo"]')]
    years_list = []
    for year in years:
        y = pattern.search(year)
        if y == None:
            years_list.append('none')
        else:
            years_list.append(y.group())
    #查找所需二手房信息，并将查找到的数据放到字典中       
    item = {}
    item['Renovation'] = [i.get_text().strip().split('|')[4] for i in soup.select('div[class="houseInfo"]')] #改造情况
    item['Garden'] = [i.get_text().strip().split('|')[0] for i in soup.select('div[class="houseInfo"]')]    #  房名
    item['Layout'] = [i.get_text().strip().split('|')[1] for i in soup.select('div[class="houseInfo"]')]    #户型
    item['Size'] = [i.get_text().strip().split('|')[2] for i in soup.select('div[class="houseInfo"]')]      #大小
    item['Direction'] = [i.get_text().strip().split('|')[3] for i in soup.select('div[class="houseInfo"]')] #朝向
    item['Elevator'] = [i.get_text().strip().split('|')[-1] for i in soup.select('div[class="houseInfo"]')] #电梯情况
    item['Year'] = years_list                                                                               #二手房年份
    item['Distric'] = [i.get_text().strip().split('-')[1] for i in soup.select('div[class="positionInfo"]')] # 小区
    item['Floor'] = [i.get_text().strip().split('-')[0][0:3] for i in soup.select('div[class="positionInfo"]')] #楼层
    item['Price'] = [i.get_text().strip() for i in soup.select('div[class="totalPrice"] span')]                 #价格
    item['house_unit_price'] = [i.get_text().strip() for i in soup.select('div[class="unitPrice"] span')]       #均价
    return pd.DataFrame(item)
    
                
               
    

In [4]:
#获取前100页二手房信息，并进行保存成文件
def main():
    filename = r'C:\Users\51429\Desktop\传胜学习资料\wuhanlianjia.csv'
    house_data = []
    for pg in range(1,101):
        lianjia_url = 'https://wh.lianjia.com/ershoufang/pg' + str(pg) + '/'
        page = get_page(lianjia_url,pg) 
        if len(page) > 0:
            house_info =  parse_page(page)            
            house_data.append(house_info)
        time.sleep(3)    
    data = pd.concat(house_data, ignore_index = True)
    data.to_csv(filename,encoding = 'gbk', index = False)    
    print('------写入完毕------')

        

In [5]:
main()

--------第1页抓取成功--------
--------第2页抓取成功--------
--------第3页抓取成功--------
--------第4页抓取成功--------
--------第5页抓取成功--------
--------第6页抓取成功--------
--------第7页抓取成功--------
--------第8页抓取成功--------
--------第9页抓取成功--------
--------第10页抓取成功--------
--------第11页抓取成功--------
--------第12页抓取成功--------
--------第13页抓取成功--------
--------第14页抓取成功--------
--------第15页抓取成功--------
--------第16页抓取成功--------
--------第17页抓取成功--------
--------第18页抓取成功--------
--------第19页抓取成功--------
--------第20页抓取成功--------
--------第21页抓取成功--------
--------第22页抓取成功--------
--------第23页抓取成功--------
--------第24页抓取成功--------
--------第25页抓取成功--------
--------第26页抓取成功--------
--------第27页抓取成功--------
--------第28页抓取成功--------
--------第29页抓取成功--------
--------第30页抓取成功--------
--------第31页抓取成功--------
--------第32页抓取成功--------
--------第33页抓取成功--------
--------第34页抓取成功--------
--------第35页抓取成功--------
--------第36页抓取成功--------
--------第37页抓取成功--------
--------第38页抓取成功--------
--------第39页抓取成功--------
--------第40页抓取成功--------
--------第

In [13]:
f = open(r'C:\Users\51429\Desktop\传胜学习资料\wuhanlianjia.csv','r')
wuhanlianjia_df = pd.read_csv(f)
wuhanlianjia_df.head()

,Renovation,Garden,Layout,Size,Direction,Elevator,Year,Distric,Floor,Price,house_unit_price
0,精装,星海虹城一期,4室2厅,144.33平米,东 南 西,有电梯,2006,楚河汉街,低楼层,440.0,单价30486元/平米
1,其他,保利中央公馆,3室2厅,116.83平米,南 北,有电梯,2011,新南湖,中楼层,265.0,单价22683元/平米
2,其他,金色雅园金源苑,2室2厅,102.74平米,南,无电梯,2002,长港路,中楼层,175.0,单价17034元/平米
3,简装,常青花园十一小区,2室2厅,91.57平米,南,有电梯,2008,常青花园,低楼层,198.0,单价21623元/平米
4,精装,保利海上五月花一期,3室2厅,86.8平米,南,有电梯,none,庙山,高楼层,140.0,单价16130元/平米


In [14]:
f.close()

通过浏览csv文件发现一些数据特征存在误差，后续将进一步对数据进行清洗整理，并对整理好的数据来进行可视化展示，分析二手房房价与地区，年份，楼层等特征之间关系，以及武汉各区二手房售卖数量进行总结